In [1]:
import os
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"
os.environ["VLLM_USE_FLASHINFER"] = "0"
os.environ["VLLM_ATTENTION_BACKEND"] = "TRITON_ATTN"

In [2]:
!env | grep PYTORCH_CUDA_ALLOC_CONF
!env | grep VLLM_USE_FLASHINFER

VLLM_USE_FLASHINFER=0


In [ ]:
!pip install -U numpy==1.26.4 scikit-learn==1.5.2 --no-cache-dir
!pip install transformers==4.56.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 238.3 MB/s eta 0:00:000:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 100.8 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.

In [ ]:
!pip install -q --upgrade pip
!pip install -q vllm==0.11.1 
!pip install -q huggingface_hub
!pip install flask pyngrok flask_cors
!pip install -q bitsandbytes accelerate
# print("--- [FIX] Bắt đầu sửa lỗi xung đột numpy/sklearn ---")
# # # Buộc cài đặt lại các gói bị ảnh hưởng để đảm bảo chúng tương thích
# !pip install --upgrade --force-reinstall scikit-learn
# 
# print("--- [FIX] Đã hoàn tất sửa lỗi ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:0000:010:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cudf-cu12 25.2.2 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0

In [5]:
!rm -rf /kaggle/working/*

In [6]:
import subprocess
import time
import requests
import re
from pyngrok import ngrok
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from huggingface_hub import snapshot_download
import json
import os
import requests
import random
from openai import OpenAI
from typing import List, Set, Dict, Tuple, Optional

In [7]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")
if hf_token is None:
    raise RuntimeError("Thiếu HF_TOKEN. Vui lòng thêm vào Kaggle Secrets.")

In [8]:
login(token=hf_token)
print("✅ Đăng nhập Hugging Face thành công!")
# Ô 2b: Tải model safetensors từ repo gated


# MODEL_REPO = "ktam204/Qwen3-32B-AWQ-r16-lora-all-Pentest-swift"
# MODEL_REPO = "Qwen/Qwen3-32B-AWQ"
# LOCAL_DIR   = "/kaggle/tmp/Qwen3-32B"

# MODEL_REPO = "cpatonn/Qwen3-Coder-30B-A3B-Instruct-AWQ-4bit"
MODEL_REPO = "Qwen/Qwen2.5-32B-Instruct-AWQ"
# MODEL_REPO = "Qwen/Qwen3-4B-Instruct-2507"
LOCAL_DIR = "/kaggle/tmp/Qwen2.5-32B-Instruct-AWQ"


snapshot_download(
    repo_id=MODEL_REPO,
    local_dir=LOCAL_DIR,
    local_dir_use_symlinks=False
)
print("✅ Đã download xong vào:", LOCAL_DIR)

✅ Đăng nhập Hugging Face thành công!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

✅ Đã download xong vào: /kaggle/tmp/Qwen2.5-32B-Instruct-AWQ


In [9]:
!pip cache purge

Files removed: 450 (4819.4 MB)


In [10]:
# !ls $LOCAL_DIR

In [11]:
# !cat /usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py

In [ ]:
# Log file names
vllm_logfile = 'vllm.log'

# 1) Launch vLLM server in background
# vllm_cmd = [
#     "python3", "-m", "vllm.entrypoints.openai.api_server",
#     "--model", LOCAL_DIR,
#     "--served-model-name", "Qwen3-32B",
#     "--dtype", "float16",             # ← use float16 on L4
#     "--gpu-memory-utilization", "0.9",
#     "--tensor-parallel-size", "4",
#     "--enforce-eager",
#     "--max-model-len", "16384",
#     # "--max-num-seqs", "1024",
#     #"--enable-reasoning",
#     # "--reasoning-parser", "deepseek_r1",
#     "--enable-auto-tool-choice", 
#     "--tool-call-parser", "hermes",
#     "--host", "0.0.0.0",
#     "--port", "8000",
# ]

vllm_cmd = [
    "python3", "-m", "vllm.entrypoints.openai.api_server",
    "--model", LOCAL_DIR,#"/kaggle/input/qwen-3/transformers/32b/1",
    "--served-model-name", "Qwen2.5-32B-Instruct-AWQ",
    "--dtype", "float16",             # ← use float16 on T4
    "--gpu-memory-utilization", "0.9",
    "--tensor-parallel-size", "2",
    "--enforce-eager",
    "--max-model-len", "16384",
    # "--reasoning-parser", "deepseek_r1",
    # "--enable-auto-tool-choice", 
    # "--tool-call-parser", "hermes",
    "--host", "0.0.0.0",
    "--port", "8000",
]

with open(vllm_logfile, "w") as logf:
    proc = subprocess.Popen(vllm_cmd, stdout=logf, stderr=subprocess.STDOUT)

# 2) Poll /v1/models up to 360 times, waiting 10s between tries (max 1 hour)
for attempt in range(360):
    try:
        resp = requests.get("http://127.0.0.1:8000/v1/models", timeout=10)
        if resp.status_code == 200:
            print(f"✅ vLLM is up on attempt {attempt+1}!")
            break
    except requests.RequestException as e:
       print(f"Error on attempt {attempt+1}: {e}")
       with open(vllm_logfile, "r") as f:
           lines = f.readlines()
       print("📄 Last 20 lines of log:")
       print("".join(lines[-50:]))
    print(f"Waiting for vLLM startup... ({attempt+1}/360)")
    time.sleep(10)
else:
    print("❌ Still not up after 1 hour. Check", vllm_logfile)
    proc.terminate()
    raise RuntimeError("vLLM startup timeout")

Error on attempt 1: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a2404cd190>: Failed to establish a new connection: [Errno 111] Connection refused'))
📄 Last 20 lines of log:

Waiting for vLLM startup... (1/360)
Error on attempt 2: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a2404dd450>: Failed to establish a new connection: [Errno 111] Connection refused'))
📄 Last 20 lines of log:

Waiting for vLLM startup... (2/360)
Error on attempt 3: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a25b7eb310>: Failed to establish a new connection: [Errno 111] Connection refused'))
📄 Last 20 lines of log:
2025-11-28 08:03:10.113769

In [ ]:
secrets = UserSecretsClient()
ngrok.set_auth_token(secrets.get_secret("ngrok_key"))

# 3) Open an HTTP tunnel on port 8000
tunnel = ngrok.connect(8000, "http")
public_url = tunnel.public_url
# 4) Print the public URL
print("🚀 Public URL for your vLLM API:", public_url)



🚀 Public URL for your vLLM API: https://suboral-gertha-autoicous.ngrok-free.dev                    


In [14]:
 # !curl http://localhost:8000/v1/models

In [ ]:
openAI_key = secrets.get_secret("OpenAI_key")
api_base = public_url.rstrip("/") + "/v1" 
client = OpenAI(base_url=api_base, api_key=openAI_key)
models = requests.get(f"{api_base}/models").json()["data"]
model_id = models[0]["id"]
print("🚀 Public URL for your vLLM API:", api_base)
print(model_id)

🚀 Public URL for your vLLM API: https://suboral-gertha-autoicous.ngrok-free.dev/v1
Qwen2.5-32B-Instruct-AWQ


In [ ]:
import time

print("🔄 BẮT ĐẦU KEEP-ALIVE LOOP")
print("⚠️  ĐỪNG ĐÓNG TAB NÀY! Để giữ session Kaggle sống.\n")

while True:
    print("⚙️ Still running...")
    time.sleep(60)
